In [1]:
import pandas as pd
import os, sys
import numpy as np


In [2]:
os.chdir('C:/Users/euna/Desktop/metis')
sys.path.append('C:/Users/euna/Desktop/metis')

In [3]:
arts_df = pd.read_csv('C:/Users/euna/Desktop/metis/arts/arts.csv')

C:\Users\euna\AppData\Local\Temp\ipykernel_13348\167105501.py:1: DtypeWarning: Columns (2,11,15,16,17,18,20,21,22,25,26,27,28,29,31,32,34,35,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  arts_df = pd.read_csv('C:/Users/euna/Desktop/metis/arts/arts.csv')


In [4]:
#col 이름 확인 
arts_df.columns

Index(['Unnamed: 0', 'overall', 'vote', 'verified', 'reviewTime', 'reviewerID',
       'asin', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'style.Format:', 'image', 'style.Color:', 'style.Size:',
       'style.Size Name:', 'style.Style:', 'style.Style Name:',
       'style.Color Name:', 'style.Package Quantity:', 'style.Package Type:',
       'style.Design:', 'style.Pattern:', 'style.Item Package Quantity:',
       'style.Item Display Length:', 'style.Length:',
       'style.Product Packaging:', 'style.Metal Type:', 'style.style name:',
       'style.Material Type:', 'style.Display Height:', 'style.Edition:',
       'style.Material:', 'style.Number of Items:', 'style.Scent:',
       'style.Scent Name:', 'style.Overall Length:', 'style.Shape:',
       'style.Item Display Weight:', 'style.Team Name:', 'style.Flavor:'],
      dtype='object')

In [5]:
#필요한 열만 사용
arts_df = arts_df[['reviewTime', 'reviewerID', 'asin']]

In [6]:
# review 3개 이상 한 사람의 세션 길이 구하려고
test_df = arts_df.groupby('reviewerID').count()
test_df = test_df[test_df['asin']>2]

In [7]:
test_df

,reviewTime,asin
reviewerID,,
A00181966XZUX7KEGYLH,3,3
A00327403XHFRKYW3TDN,3,3
A00408825PVJW7GFLEGU,6,6
A0045558RLEOANWJ9H6A,3,3
A0049438OU67GPDVNT4Y,3,3
...,...,...
AZZX1X30TFZZI,4,4
AZZXGHDZV05KT,3,3
AZZXW4PH4OEEK,19,19


In [8]:
#session길이를 만드는 중
test_df.rename(columns={'asin' : 'sessionlen'}, inplace=True)

In [9]:
test_df.reset_index(drop=False)

,reviewerID,reviewTime,sessionlen
0,A00181966XZUX7KEGYLH,3,3
1,A00327403XHFRKYW3TDN,3,3
2,A00408825PVJW7GFLEGU,6,6
3,A0045558RLEOANWJ9H6A,3,3
4,A0049438OU67GPDVNT4Y,3,3
...,...,...,...
223507,AZZX1X30TFZZI,4,4
223508,AZZXGHDZV05KT,3,3
223509,AZZXW4PH4OEEK,19,19
223510,AZZYCJOJLUDYR,5,5


In [10]:
#세션안에서 시간순으로 정렬해주기위해 리뷰 시간을 datetime으로 바꿈
arts_df['reviewTime'] = pd.to_datetime(arts_df['reviewTime'])

In [11]:
df_sorted=arts_df.sort_values(by=['reviewerID','reviewTime'])

In [13]:
df_merged = pd.merge(df_sorted, test_df, how='outer', on='reviewerID')

In [15]:
df_merged = df_merged[df_merged['sessionlen']>2]
df_merged = df_merged.reset_index(drop=True)

In [16]:
df_merged

,reviewTime_x,reviewerID,asin,reviewTime_y,sessionlen
0,2017-02-10,A00181966XZUX7KEGYLH,B01BW225Q6,3.0,3.0
1,2017-09-29,A00181966XZUX7KEGYLH,B000IYXIZM,3.0,3.0
2,2017-09-29,A00181966XZUX7KEGYLH,B01C9UYT46,3.0,3.0
3,2017-07-22,A00327403XHFRKYW3TDN,B0015590VO,3.0,3.0
4,2017-07-22,A00327403XHFRKYW3TDN,B00HNYHKIQ,3.0,3.0
...,...,...,...,...,...
1265790,2016-10-20,AZZYCJOJLUDYR,B001QX2G9S,5.0,5.0
1265791,2016-10-21,AZZYCJOJLUDYR,B005JGZEHO,5.0,5.0
1265792,2016-05-30,AZZZPAB0MBZ69,B00VUXT4RQ,3.0,3.0
1265793,2016-05-30,AZZZPAB0MBZ69,B014JU3BGG,3.0,3.0


In [17]:
#세션 아이티와 아이템 아이디를 붙여줌
unique_arr=df_merged['reviewerID'].unique()
unique_item=df_merged['asin'].unique()

id_map={key:i for i, key in enumerate(unique_arr)}
df_merged['sessionId'] = df_merged['reviewerID'].map(id_map)

id_map={key:i for i, key in enumerate(unique_item)}
df_merged['itemId'] = df_merged['asin'].map(id_map)

In [19]:
df_merged.rename(columns={'reviewTime_x': 'reviewTime'}, inplace=True)

In [20]:
df_merged = df_merged[['reviewTime', 'reviewerID', 'asin', 'sessionlen', 'sessionId', 'itemId']]

In [22]:
test_df = df_merged.groupby('reviewerID').count()
test_df = test_df[['sessionlen']]
test_list = list(test_df['sessionlen'])

In [23]:
val = 0
len_list = []
for i,idx in enumerate(test_list):
    for j in range(idx):
        len_list.append(val)
        val += 1
    val = 0



In [24]:
from datetime import timedelta

In [25]:
df_merged['a'] = len_list

In [26]:
df_merged['b'] = df_merged.a*timedelta(seconds=30)

In [28]:
df_merged['datetime'] = df_merged['reviewTime'] + df_merged['b']

In [30]:
df_merged['timestamp'] = pd.to_datetime(df_merged['datetime']).astype(np.int64) / 10**9

In [32]:
df_merged = df_merged[['reviewerID', 'asin', 'itemId', 'sessionId', 'timestamp']]

In [34]:
def num(train_sess_num, train_ratio=0.8, val_ratio=0.1):
    
    train_num = train_sess_num*train_ratio
    val_num = train_num + train_sess_num*val_ratio


    return train_num, val_num, train_sess_num


a,b,c = num(25000)


df_train = df_merged[df_merged['sessionId'] < a]
df_val = df_merged[(df_merged['sessionId'] >= a) & (df_merged['sessionId'] < b)]
df_test = df_merged[(df_merged['sessionId'] >= b) & (df_merged['sessionId'] < c)]
df_all = df_merged[df_merged['sessionId'] < c]



In [36]:
def filter_less_more(df, min_len, max_len):
    less_more = df.groupby('reviewerID').size()
    less_more_idx = less_more[(less_more >= min_len) & (less_more <= max_len)].index
    less_more_df = df[df['reviewerID'].isin(less_more_idx)]

    cut = df.groupby('reviewerID').size()
    cut_idx = cut[cut > max_len].index
    cut_df = df[df['reviewerID'].isin(cut_idx)].groupby('reviewerID').tail(50)

    filter_df = pd.concat([less_more_df, cut_df], axis=0)

    return filter_df

min_len = 4
max_len = 50
train_filter = filter_less_more(df_train, min_len, max_len)
val_filter = filter_less_more(df_val, min_len, max_len)
test_filter = filter_less_more(df_test, min_len, max_len)